In [1]:
import math
import tsplib95
import statistics
import numpy as np
from itertools import groupby
from scipy.special import _logsumexp

In [2]:
def first_index(arr: list) -> float:
    if (len(arr) == 1):
        return arr
    else:
        return arr[0]

In [35]:
def decide_city(Vt: float, acc: float, time: float, eye: int, n: int) -> int:
    city = 0
    if eye == 1:
        divided_value = int(Vt * np.nan_to_num(time) - (1/2) * acc * (np.power(np.nan_to_num(time), 2)))
    else:
        divided_value = int(Vt * np.nan_to_num(time) + (1/2) * acc * (np.power(np.nan_to_num(time), 2)))
        
    if divided_value > (n - 1):
        city = divided_value - (int(divided_value / n) * n) 

    return city

### Normal functions of the algorithm

In [36]:
def herding(route: list, Vt: list, fit: list, n: int, dimension: int, acc: list, time: list) -> list:
    
    route_copy = np.array(route)
    fit1 = np.sort(fit, 0, kind="heap")
    idx = np.argsort(fit, 0, kind="heap")
    route1 = np.zeros(len(route))
    Vt1 = np.zeros((n, dimension))
    acc1 = np.zeros((n, dimension))
    time1 = np.zeros((n, dimension))

    for i in range(n):
        route1[i] = first_index(route_copy[idx[i]])
        Vt1[i, :] = Vt[idx[i][0], :]
        acc1[i, :] = acc[i, :]
        time1[i] = time[i]
    
    return route1, Vt1, fit1, acc1, time1

In [37]:
def update_v(Vt: list, n: int, dimension: int, acc: list, time: list, route: list, fit: list, eye: int) -> list:
    #Choosing left and right dogs
    right_dog = np.random.randint(2,3)
    if (right_dog == 2):
        left_dog = 3
    else:
        left_dog = 2

    acc1 = np.ones((n,dimension))
    time1 = np.ones((n,1))
    Vt1 = np.copy(Vt)
    r = len(acc)
    l = len(acc)
    acc2 = np.zeros((r, l))
    fit1 = fit[l-1]

    #Finding Dg value to choose which sheep to gather and which to stalk
    fit2 = (fit[1] + fit[2]) / 2
    f = 0
    tempg = 0
    temps = 0

    #Setting parameters for eyeing
    if eye == 1:
        if fit[right_dog] < fit[left_dog]:
            acc2[left_dog, :] = -1 * acc[left_dog, :]
            f = left_dog
        else:
            acc2[right_dog, :] = -1 * acc[right_dog, :]
            f = right_dog

    for i in range(n):
        for j in range(dimension):
            #Velocity updation of dogs
            if (i <= 2):
                Vt1[i][j] = math.sqrt(np.power(Vt[i][j], 2) + (2 * acc[i][j]) * abs(route[i]))
                
            #Velocity updation of sheep
            if (i > 2):
                if eye == 1:
                    Vt1[i][j] = math.sqrt(np.power(Vt1[f][j], 2) + (2*acc2[f][j]) * abs(route[i]))

                else:
                    #Velocity updation of gathered sheep
                    if ((fit1 - fit[i]) > (fit2 - fit[i])):
                        Vt1[i][j] = math.sqrt(np.power(Vt1[0,j], 2) + (2*acc[0,j]))
                        #tempg[i] = i        
                
                    #Velocity updation of stalked sheep
                    if ((fit1 - fit[i]) <= (fit2 - fit[i])):
                        Vt1[i][j] = math.sqrt(np.power(Vt[right_dog][j]*math.tan(np.random.randint(1,89)), 2) + (2*acc[right_dog][j]*abs(route[right_dog]))) + math.sqrt(np.power(Vt[left_dog][j]*math.tan(np.random.randint(91,179)), 2) + (2*acc[left_dog][j]*abs(route[left_dog])))
                        Vt1[i][j] = (Vt1[i][j]) / 2
                        #temps[i] = i
    
    #Updation of time and acceleration
    for i in range(n):
        s = 0
        for j in range(dimension):
            acc1[i][j] = abs((Vt1[i][j]) - (Vt[i][j])) / (time[i][j])
            s += (_logsumexp.logsumexp(Vt1[i][j]) - _logsumexp.logsumexp(Vt[i][j])) / acc1[i][j]
            
        time1[i] = abs(np.mean(s))

    return Vt1, acc1, time1, right_dog, left_dog, tempg, temps

In [38]:
def check(route: list, n: int, dimension: int, acc: list, Vt: list, time: list, objf: object) -> list:
    route1 = np.copy(route)
    acc1 = np.copy(acc)
    time1 = np.copy(time)
    Vt1 = np.copy(Vt)
    for i in range(n):
        for j in range(dimension):
            if (route[i] >= len(objf[0]) or route[i] <= np.min(objf[0])):
                route1[i] = np.random.randint(np.min(objf[0]), len(objf[0]))
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()
    
    for i in range(n):
        for j in range(dimension):
            if (math.isnan(acc[i][j]) == 1 or acc[i][j] == 0):
                route1[i] = np.random.randint(np.min(objf[0]), len(objf[0]))
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()

    for i in range(n):
        for j in range(dimension):
            if (math.isnan(Vt[i][j]) == 1 or Vt[i][j] == 0):
                route1[i] = np.random.randint(np.min(objf[0]), len(objf[0]))
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()

    for i in range(n):
        for j in range(dimension):
            if (math.isnan(time1[i]) == 1 or time1[i] == 0):
                route1[i] = np.random.randint(np.min(objf[0]), len(objf[0]))
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()


    return route1, acc1, time1, Vt1

### Discretization of the problem

In [39]:
def discrete_fitness(route: list, n: int, objf: object):
    fit1 = np.zeros((len(route), 1))

    for i in range(len(route)):    
        if (i != len(route) - 1):
            fit1[i] = objf[int(route[i])][int(route[i+1])]
        else:            
            fit1[i] = objf[int(route[i])][int(route[0])]
    
    #best fit and position
    best_fit = np.min(fit1)
    pos = np.argmin(fit1)
    
    return fit1, best_fit, pos 

In [40]:
def discrete_generate(n: int, dimension: int) -> list:
    boundary_no = 1
    pop = np.zeros((n, dimension))

    if boundary_no == 1:
        pop = np.random.randint(0, 1+1, size=(n,dimension))

    x = np.random.random(size = (n, dimension))
    
    return pop, x

In [41]:
def discrete_update(Vt: list, time: list, acc: list, n: int, dimension: int, eye: int) -> list:
    route1 = np.zeros((n, dimension))
    for i in range(n):
        for j in range(dimension):
            #Updating the position of dogs
            # Check the min and max value of each variable in the equation
            if (i <= 2):
                route1[i] = decide_city(Vt[i][j], acc[i][j], time[i], eye, n)
                
            #Updating position of sheep
            if (i > 2):
                if eye == 1:
                    route1[i] = decide_city(Vt[i][j], acc[i][j], time[i], eye, n)
                else:
                    route1[i] = decide_city(Vt[i][j], acc[i][j], time[i], eye, n)    

    return route1

### Funtions to define  the route

In [42]:
# Need to put at least one sheep per city
# Create a function to make the reward strategy

def city_choice(n: int) -> list:
    route = []
    city = []
    sheep = np.random.randint(0, n)

    for map in range(n):
        city.append(map)
    
    while len(city) != 0:
        if sheep not in city:
            sheep = np.random.randint(0, n)
        else:
            route.append(sheep)
            city.remove(sheep)

    return route

# Create a random number each iteration and after that made a line of the matrix receive that number
def generate_city(n: int, dim: int) -> list:
    city_matrix = np.zeros((n, dim))
    sheep = city_choice(n)

    for i in range(n):
        for j in range(dim):
            if (sheep[i] == j):
                city_matrix[i][j] = 1
    
    x = np.random.random(size = (n, dim))
    
    return city_matrix, x

# Debug function
# There's no need to use this function in the main algorithm, just in the test file
def print_position_of_sheep(city_matrix: list):
    for i in range(len(city_matrix)):
        for j in range(len(city_matrix)):
            if (city_matrix[i][j] != 0):
                print("Position -> ",j)

# Function to create route
def create_route(city_matrix: list) -> list:
    route = []
    for i in range(len(city_matrix)):
        for j in range(len(city_matrix)):
            if (city_matrix[i][j] == 1):
                route.append(j)
    print("Route -> ",route,"\n")

    return route

# Function to verify route
def verify_route(route: list) -> bool:
    # Append each position in a list, after that verify if the route is valid in a different ways
    valid = True
    for i in range(len(route)):
        if (i == (-1) and route[i] != 0):
            valid = False
    valid = same_cities(route)

    return valid

def same_cities(route: list) -> bool:
    valid = True
    aux = route[-1]
    i = 1
    j = 1
    while i != (len(route) - 2):
        if (aux == route[j]):
            valid = False
            break
        j += 1
        if (j == (len(route) - 1)):
            aux = route[i]
            i += 1
            j = 1

    return valid    

def punishment(city_matrix: list, route: list, weight: int) -> list:
    aux_verify = verify_route(route)
    aux_route = same_cities(route)
    position = route[1]
    if (aux_route == True or aux_verify == False):
        city_matrix[0][int(position)] *= np.power(weight, 2)
    
    # print("Route -> ",route,"\n")
    # print("City matrix -> ",city_matrix,"\n")
    
    return city_matrix

def catch_city(arr: list, n: int) -> list:
    route = np.zeros(n)
    for i in range(len(arr)):
        route[i] = arr[i][0]
    
    return route

### Application of TSP

In [43]:
problem = tsplib95.load('../data/domain/bays29.tsp')

In [44]:
#Maximum no. of iterations
gen = 200

#Population size
n = problem.dimension

#Optimization function name
dim = problem.dimension

#Intialize the population(init_p-Population,acc-acceleration of each individual)
init_p, acc = generate_city(n, dim)

#Vt = velocity of each individuals
Vt = np.zeros((n, dim))

#Time of each individual
time = np.random.random(size = (n,1))

#Max fitness value
fopt = math.inf
#Variable to store fitness
fit = np.zeros((n, 1))
pop = init_p
#print_position_of_sheep(pop)
#k = counter variable for iterations required for Eyeing mechanism
k = 1
fopt_1 = np.zeros((gen, 1))
# Generate a route and at the same time verify if this route is valid
# Needs to generate before the loop and adjust in till the iterations turns it valid
route = create_route(pop)

for g in range(gen):
    valid = verify_route(route)
    # Calculate fitness of indivuals
    # Need to remember to debug this function
    fit, maxf, pos = discrete_fitness(route, n, getattr(problem, 'edge_weights'))
    #print("Valid -> ", valid,"\n")
    eye = 0
    if g == 0:
        fopt = maxf
    
    #Finding the optimum fitness value
    if fopt > maxf:
        fopt = maxf
    
    fopt_1[g] = fopt
    if g > 0:
        if fopt_1[g] > fopt_1[g-1]:
            k += 1
            if k > 5:
                eye = 1
                k = 0
    
    route, Vt, fit, acc, time = herding(route, Vt, fit, n, dim, acc, time)
    
    Vt, acc, time, r1, l1, tempg, temps = update_v(Vt, n, dim, acc, time, route, fit, eye)
    
    route = discrete_update(Vt, time, acc, n, dim, eye)
    # Do a function to catch the first index of the array
    route = catch_city(route, n)
    # Checking if the range of the population is maintained
    route, acc, time, Vt = check(route, n, dim, acc, Vt, time, getattr(problem, 'edge_weights'))
    
    print("Route -> ",route,"\n")

Route ->  [7, 2, 24, 14, 16, 19, 27, 0, 8, 11, 6, 22, 17, 15, 3, 12, 10, 25, 23, 21, 26, 5, 13, 9, 4, 1, 28, 18, 20] 

Route ->  [22.  7.  5. 21. 19. 15.  6. 18. 28.  3.  1.  3. 16. 28. 21. 18. 27.  7.
 24. 26. 10. 25. 22. 12. 22.  9. 10. 14.  2.] 

Route ->  [21.  6. 15.  4. 17.  1. 15. 20.  4. 28. 17. 11. 18. 13. 19.  9. 16. 19.
 18. 17.  2. 14. 22. 13. 18.  9.  2. 11. 19.] 

Route ->  [14. 14. 19.  6. 12.  7. 24.  4. 26. 18. 24. 25. 17.  7. 24. 20. 26. 25.
  2. 14. 10.  4.  7. 17.  2. 20. 26.  9.  5.] 

Route ->  [ 1. 10.  8. 10. 12. 25. 22. 21. 17. 19. 14.  1.  1. 28.  8. 24. 28. 27.
  1.  6. 28. 24. 23. 11.  8. 24. 20. 13.  3.] 

Route ->  [22. 27. 25.  7. 15. 13. 21.  9. 23. 10. 21. 13. 17. 18.  3. 20. 25.  1.
 20.  9. 10. 17.  6. 21. 26.  6. 20. 13.  3.] 

Route ->  [ 9.  5.  1.  4. 12. 15.  7. 25. 18.  1. 28.  4.  4. 10. 21. 25.  1. 10.
 14. 26.  2. 20. 25. 14.  9.  8. 22.  6.  7.] 



C:\Users\artur\AppData\Local\Temp/ipykernel_300/3957245458.py:60: RuntimeWarning: invalid value encountered in double_scalars
  s += (_logsumexp.logsumexp(Vt1[i][j]) - _logsumexp.logsumexp(Vt[i][j])) / acc1[i][j]


Route ->  [27. 24. 26. 23. 22. 17.  3. 14. 15. 19. 10. 23. 17. 13.  9. 22. 17. 10.
 22. 11.  1. 15.  5. 26.  2. 13.  0. 16. 23.] 

Route ->  [24. 11. 20. 18. 24. 22. 23.  6.  6. 15. 12. 14. 15. 15. 25. 11. 22. 26.
 28. 26.  3. 25. 18. 25. 10. 12.  4.  2.  2.] 

Route ->  [13. 21. 18. 14. 11. 19.  2.  9. 19. 22. 15. 26. 12. 17.  4. 26. 22. 14.
 15. 25. 18. 11.  9. 14. 20. 24.  2. 13.  5.] 

Route ->  [12. 23.  7. 11.  5. 21. 12. 27. 26.  7. 28.  4. 23. 27.  2.  9. 14.  4.
 27.  6.  6. 11. 21.  1. 25. 19. 26. 19. 15.] 

Route ->  [11. 22. 11.  1. 18. 10. 12. 12.  1.  3. 17.  2. 26.  1.  4. 27. 23. 20.
 13. 15. 18. 20. 15. 14. 11. 15.  2. 24. 16.] 

Route ->  [ 1. 20. 28. 19. 18.  7. 20.  5.  5. 16. 25.  5. 11. 11.  7. 17.  7.  1.
 15. 21.  6. 22. 21.  2. 13. 13. 23. 24. 26.] 

Route ->  [ 6. 13. 27.  8.  7.  1. 10. 19.  4. 22. 16. 25. 15. 21. 13. 17.  4. 11.
 22. 15. 11. 28. 14. 16.  0. 20.  8. 18. 12.] 

Route ->  [18.  1. 20. 27.  2. 14.  3.  5.  6. 28. 19.  4. 27. 22. 13.  4. 26. 12.
